<a href="https://colab.research.google.com/github/yxpx/google_colab/blob/main/pypal_insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd

file_name = '/content/sample_data/pypal_data.xlsx'
try:
    if file_name.endswith('.csv'):
        df = pd.read_csv(file_name)
    else:
        df = pd.read_excel(file_name)

    print(f"Successfully loaded '{file_name}'.")
    print("Here are the first 5 rows of your data:")
    print(df.head(10))

except FileNotFoundError:
    print(f"--- ERROR ---")
    print(f"File not found: '{file_name}'.")

except Exception as e:
    print(f"An error occurred: {e}")

Successfully loaded '/content/sample_data/pypal_data.xlsx'.
Here are the first 5 rows of your data:
   concept_number  question_number  \
0               5                6   
1               1                1   
2               1                1   
3               1                1   
4               1                2   
5               1                2   
6               3                3   
7               1                2   
8               1                1   
9               2                2   

                                            question  \
0  Given a dictionary containing the student name...   
1  Given an integer, return True if it is even, e...   
2  Given an integer, return True if it is even, e...   
3  Given an integer, return True if it is even, e...   
4         Given two integers, return the larger one.   
5         Given two integers, return the larger one.   
6  Given a list of numbers, return their mean and...   
7         Given two integers, ret

In [28]:
import pandas as pd
import numpy as np

if 'df' in locals():
    feedback_str = df['exec_feedback'].astype(str)
    df['status'] = np.where(
        feedback_str.str.contains('error', case=False, na=False),
        'Error',
        'Fail'
    )

    print("Here is the breakdown of all problems:")
    print(df['status'].value_counts())
    print("\n")

else:
    print("--- ERROR --- 'df' is not loaded")

Here is the breakdown of all problems:
status
Fail     9141
Error    5267
Name: count, dtype: int64




In [43]:
if 'df' in locals():
    # This regex pattern looks for words that end in 'Error' or 'error'
    regex_pattern = r'([\w\s]+[Ee]rror)'

    # Use .str.extract to pull out the first match from 'exec_feedback'
    extracted_errors = df['exec_feedback'].astype(str).str.extract(regex_pattern, expand=False)

    # Create the new 'error_type' column ONLY if the status is 'Error'
    df['error_type'] = np.where(
        df['status'] == 'Error',
        extracted_errors,  # Put the extracted error text
        np.nan             # Otherwise, leave it blank
    )

    print("Most common errors are:")
    print(df['error_type'].value_counts(dropna=True).head(20))
else:
    print("--- ERROR --- 'df' is not loaded.")

Most common errors are:
error_type
 TypeError            2544
 NameError            1247
 ZeroDivisionError     607
 UnboundLocalError     390
 AttributeError        210
 EOFError               88
 IndexError             81
 ValueError             70
 KeyError               16
 RecursionError          5
Unexpected error         4
 Error                   3
 OverflowError           2
Name: count, dtype: int64


In [30]:
import pandas as pd

if 'df' in locals() and 'status' in df.columns:
    # Create the unique ID
    # We convert the numbers to strings so we can join them with a dash
    df['unique_question_id'] = 'C' + df['concept_number'].astype(str) + \
                               '-Q' + df['question_number'].astype(str)

    # Count 'Error' and 'Fail' for each unique question
    question_summary = df.groupby('unique_question_id')['status'].value_counts()

    # .unstack(fill_value=0) moves 'Error' and 'Fail' into their own columns
    question_summary_table = question_summary.unstack(fill_value=0)

    # Add a 'Total' column and sort by it
    question_summary_table['Total_Problems'] = question_summary_table.sum(axis=1)
    question_summary_table = question_summary_table.sort_values(by='Total_Problems', ascending=False)

    print("\n--- Summary of Problems by Question ---")
    print(question_summary_table.head(15))

else:
    print("--- ERROR ---")


--- Summary of Problems by Question ---
status              Error  Fail  Total_Problems
unique_question_id                             
C1-Q8                 622   923            1545
C1-Q7                 247   972            1219
C4-Q9                  51   659             710
C1-Q1                 558   106             664
C3-Q7                 141   523             664
C6-Q5                 126   361             487
C2-Q1                 185   250             435
C2-Q2                  88   344             432
C3-Q3                 148   279             427
C6-Q7                  94   317             411
C1-Q4                  42   366             408
C4-Q8                  48   283             331
C1-Q9                 262    66             328
C1-Q10                 39   286             325
C4-Q2                  38   250             288


In [31]:
if 'df' in locals() and 'anon_id' in df.columns and 'status' in df.columns:
    # Group by anon_id and count the total problems
    anon_id_summary = df.groupby('anon_id').size().to_frame(name='Total_Problems')

    # Group by anon_id and status to count errors and fails
    anon_id_status_counts = df.groupby(['anon_id', 'status']).size().unstack(fill_value=0)
    anon_id_performance = anon_id_summary.join(anon_id_status_counts)
    anon_id_performance = anon_id_performance.rename(columns={'Error': 'Errors', 'Fail': 'Fails'})

    # Sort by total problems in descending order
    anon_id_performance = anon_id_performance.sort_values(by='Total_Problems', ascending=False)

    print("\n--- Performance Summary by User (anon_id) ---")
    display(anon_id_performance.head(25))
else:
    print("--- ERROR ---")


--- Performance Summary by User (anon_id) ---


,Total_Problems,Errors,Fails
anon_id,,,
c5d3396a9af56088,118,31,87
8505438e7b8eb3e6,110,50,60
f9144dfe7f18ae59,110,107,3
337467c677292c55,105,33,72
8f2be06a802927fa,103,40,63
59d7cf7e1b3e8dbd,93,39,54
584999f99ed705dc,93,35,58
5aac5c35afb68d76,91,32,59
20a2d43ee36da2aa,89,25,64


In [32]:
if 'df' in locals() and 'anon_id' in df.columns and 'error_type' in df.columns and 'status' in df.columns:
    error_df = df[df['status'] == 'Error']

    # Group by anon_id and error_type, then count occurrences
    anon_id_error_counts = error_df.groupby(['anon_id', 'error_type']).size().unstack(fill_value=0)

    print("\n--- Top Error Types per User (anon_id) ---")
    # Display the top error types for the first few users
    display(anon_id_error_counts.head(25))

else:
    print("--- ERROR ---")


--- Top Error Types per User (anon_id) ---


error_type,AttributeError,EOFError,Error,IndexError,KeyError,NameError,OverflowError,RecursionError,TypeError,UnboundLocalError,ValueError,ZeroDivisionError,Unexpected error
anon_id,,,,,,,,,,,,,
0037428e5fd3ec77,0,0,0,1,0,2,2,0,10,10,1,3,0
0066177210a38df5,0,0,0,0,0,6,0,0,21,7,0,0,0
00e795689dbd79c4,0,0,0,0,0,3,0,0,2,0,0,2,0
010c4de134a8b1f4,1,0,0,0,0,0,0,0,12,2,0,1,1
012a83026f4eb6ee,0,0,0,0,0,9,0,0,1,0,0,0,0
01a7950b5ccaeb2d,1,0,0,4,0,8,0,0,8,1,1,5,0
01d7e59bfaef7073,0,0,0,0,0,0,0,0,0,0,1,1,0
0208d5c79d7b9ffd,0,0,1,0,0,6,0,0,12,0,0,2,0
02f419b96f01073c,0,0,0,0,0,6,0,0,1,2,0,1,0


In [34]:
import re
import pandas as pd
import numpy as np

# Ensure df is loaded
if 'df' in locals():
    # Create the 'status' column
    feedback_str = df['exec_feedback'].astype(str)
    df['status'] = np.where(
        feedback_str.str.contains('error', case=False, na=False),
        'Error',
        'Fail'
    )

    # Create the 'error_type' column
    regex_pattern = r'([\w\s]+[Ee]rror)'
    extracted_errors = df['exec_feedback'].astype(str).str.extract(regex_pattern, expand=False)
    df['error_type'] = np.where(
        df['status'] == 'Error',
        extracted_errors,
        np.nan
    )

    # Data Preparation: Create a binary matrix of error occurrences
    # Filter for rows with 'Error' status
    error_df = df[df['status'] == 'Error'].copy()

    if error_df.empty:
        print("--- ERROR ---")
    else:
        # Extract all unique error types from the 'exec_feedback' column
        all_errors = set()
        for feedback in error_df['exec_feedback'].dropna():
            if isinstance(feedback, str):
                matches = re.findall(r'([\w\s]+[Ee]rror)', feedback)
                for match in matches:
                    all_errors.add(match.strip())
            elif isinstance(feedback, dict) and 'error' in feedback and isinstance(feedback['error'], str):
                 matches = re.findall(r'([\w\s]+[Ee]rror)', feedback['error'])
                 for match in matches:
                    all_errors.add(match.strip())

        if not all_errors:
            print("--- ERROR --- No error types found in 'exec_feedback' for 'Error' rows.")
        else:
            # Create a binary matrix
            error_matrix = pd.DataFrame(0, index=error_df.index, columns=list(all_errors))

            # Populate the matrix
            for index, row in error_df.iterrows():
                feedback = row['exec_feedback']
                if isinstance(feedback, str):
                     matches = re.findall(r'([\w\s]+[Ee]rror)', feedback)
                     for match in matches:
                        if match.strip() in error_matrix.columns:
                           error_matrix.at[index, match.strip()] = 1
                elif isinstance(feedback, dict) and 'error' in feedback and isinstance(feedback['error'], str):
                     matches = re.findall(r'([\w\s]+[Ee]rror)', feedback['error'])
                     for match in matches:
                        if match.strip() in error_matrix.columns:
                           error_matrix.at[index, match.strip()] = 1


            print("Binary error matrix created. Shape:", error_matrix.shape)
            display(error_matrix.head())

else:
    print("--- ERROR ---")

Binary error matrix created. Shape: (5267, 13)


,AttributeError,OverflowError,EOFError,NameError,RecursionError,Unexpected error,ValueError,TypeError,Error,UnboundLocalError,ZeroDivisionError,IndexError,KeyError
0,0,0,0,1,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,0,0,1,0,0,0,0,0
20,0,0,0,0,0,0,0,1,0,0,0,0,0
23,0,0,0,0,0,0,0,1,0,0,0,0,0
24,0,0,0,0,0,0,0,0,0,0,0,0,1


In [40]:
import pandas as pd

# Ensure df and error_matrix are available
if 'df' in locals() and 'error_matrix' in locals() and not error_matrix.empty:
    # Link error_matrix rows back to anon_id using the original index
    error_matrix_with_id = error_matrix.copy()
    error_matrix_with_id['anon_id'] = df.loc[error_matrix_with_id.index, 'anon_id']

    # Group by anon_id and sum the error occurrences to get a user's error profile
    # A value > 0 indicates the user encountered that error at least once
    user_error_profiles = error_matrix_with_id.groupby('anon_id').sum()

    # Convert sums to binary (1 if error occurred at least once, 0 otherwise)
    user_error_profiles = user_error_profiles.applymap(lambda x: 1 if x > 0 else 0)


    print("User Error Profiles created. Each row represents a user and columns indicate if an error type was encountered.")
    print("Shape:", user_error_profiles.shape)
    display(user_error_profiles.head(20))

else:
    print("--- ERROR ---")

User Error Profiles created. Each row represents a user and columns indicate if an error type was encountered.
Shape: (445, 13)


/tmp/ipython-input-3329938117.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  user_error_profiles = user_error_profiles.applymap(lambda x: 1 if x > 0 else 0)


,AttributeError,OverflowError,EOFError,NameError,RecursionError,Unexpected error,ValueError,TypeError,Error,UnboundLocalError,ZeroDivisionError,IndexError,KeyError
anon_id,,,,,,,,,,,,,
0037428e5fd3ec77,0,1,0,1,0,0,1,1,0,1,1,1,0
0066177210a38df5,0,0,0,1,0,0,0,1,0,1,0,0,0
00e795689dbd79c4,0,0,0,1,0,0,0,1,0,0,1,0,0
010c4de134a8b1f4,1,0,0,0,0,1,0,1,0,1,1,0,0
012a83026f4eb6ee,0,0,0,1,0,0,0,1,0,0,0,0,0
01a7950b5ccaeb2d,1,0,0,1,0,0,1,1,0,1,1,1,0
01d7e59bfaef7073,0,0,0,0,0,0,1,0,0,0,1,0,0
0208d5c79d7b9ffd,0,0,0,1,0,0,0,1,1,0,1,0,0
02f419b96f01073c,0,0,0,1,0,0,0,1,0,1,1,0,0


In [41]:
import itertools
import pandas as pd

# Ensure user_error_profiles is available
if 'user_error_profiles' in locals() and not user_error_profiles.empty:
    print("Analyzing frequent error combinations per user...")

    # Get the list of error types (columns in user_error_profiles)
    error_types = user_error_profiles.columns.tolist()

    # Initialize dictionaries to store counts of pairs and triplets
    pair_counts = {}
    triplet_counts = {}

    # Iterate through each user's error profile
    for index, user_profile in user_error_profiles.iterrows():
        # Get the errors encountered by this user
        user_errors = user_profile[user_profile == 1].index.tolist()

        # Find all pairs of errors for this user
        for pair in itertools.combinations(user_errors, 2):
            # Sort the pair to count combinations regardless of order
            sorted_pair = tuple(sorted(pair))
            pair_counts[sorted_pair] = pair_counts.get(sorted_pair, 0) + 1

        # Find all triplets of errors for this user
        for triplet in itertools.combinations(user_errors, 3):
             # Sort the triplet to count combinations regardless of order
            sorted_triplet = tuple(sorted(triplet))
            triplet_counts[sorted_triplet] = triplet_counts.get(sorted_triplet, 0) + 1


    # Convert counts to DataFrames and sort by frequency
    pair_counts_df = pd.DataFrame.from_dict(pair_counts, orient='index', columns=['Frequency']).sort_values(by='Frequency', ascending=False)
    triplet_counts_df = pd.DataFrame.from_dict(triplet_counts, orient='index', columns=['Frequency']).sort_values(by='Frequency', ascending=False)

    print("\n--- Most Frequent Error Pairs Across Users ---")
    display(pair_counts_df.head(15)) # Display top 15 pairs

    print("\n--- Most Frequent Error Triplets Across Users ---")
    display(triplet_counts_df.head(25)) # Display top 15 triplets

else:
    print("--- ERROR ---")

Analyzing frequent error combinations per user...

--- Most Frequent Error Pairs Across Users ---


,Frequency
"(NameError, TypeError)",275
"(TypeError, ZeroDivisionError)",208
"(NameError, ZeroDivisionError)",162
"(TypeError, UnboundLocalError)",105
"(NameError, UnboundLocalError)",98
"(AttributeError, TypeError)",83
"(AttributeError, NameError)",75
"(UnboundLocalError, ZeroDivisionError)",69
"(AttributeError, ZeroDivisionError)",49
"(TypeError, ValueError)",45



--- Most Frequent Error Triplets Across Users ---


,Frequency
"(NameError, TypeError, ZeroDivisionError)",153
"(NameError, TypeError, UnboundLocalError)",92
"(AttributeError, NameError, TypeError)",72
"(TypeError, UnboundLocalError, ZeroDivisionError)",66
"(NameError, UnboundLocalError, ZeroDivisionError)",59
"(AttributeError, TypeError, ZeroDivisionError)",48
"(AttributeError, NameError, ZeroDivisionError)",45
"(AttributeError, NameError, UnboundLocalError)",41
"(AttributeError, TypeError, UnboundLocalError)",40
"(NameError, TypeError, ValueError)",38


In [44]:
if 'df' in locals() and all(col in df.columns for col in ['status', 'unique_question_id', 'error_type', 'anon_id']):

    total_problems = len(df)
    status_counts = df['status'].value_counts()
    error_count = status_counts.get('Error', 0)
    fail_count = status_counts.get('Fail', 0)

    error_percentage = (error_count / total_problems) * 100
    fail_percentage = (fail_count / total_problems) * 100

    error_type_counts = df['error_type'].value_counts(dropna=True)
    top_3_errors = error_type_counts.head(3)
    # Calculate the percentage of the top 3 errors out of ALL errors
    top_3_errors_percentage_of_errors = (top_3_errors.sum() / error_count) * 100 if error_count > 0 else 0

    question_problems = df.groupby('unique_question_id').size().sort_values(ascending=False)
    top_6_questions = question_problems.head(6)
    top_6_questions_percentage_of_total = (top_6_questions.sum() / total_problems) * 100

    # Get data from user performance summary (assuming anon_id_performance is available)
    top_users_by_problems = None
    if 'anon_id_performance' in locals() and not anon_id_performance.empty:
        top_users_by_problems = anon_id_performance.head(5) # Top 5 users by total problems

    # Get data from frequent error pairs and triplets (assuming pair_counts_df and triplet_counts_df are available)
    most_frequent_pairs = None
    most_frequent_triplets = None
    if 'pair_counts_df' in locals() and not pair_counts_df.empty:
        most_frequent_pairs = pair_counts_df.head(5) # Top 5 frequent pairs
    if 'triplet_counts_df' in locals() and not triplet_counts_df.empty:
        most_frequent_triplets = triplet_counts_df.head(5) # Top 5 frequent triplets


    print("\n=====================================")
    print("      Comprehensive Analysis Summary")
    print("=====================================")

    print(f"\nAnalyzed a total of {total_problems} problem logs.")
    print(f"\n1. Breakdown of Problem Types:")
    print(f"   - Code-Crashing Errors: {error_count} cases ({error_percentage:.1f}%)")
    print(f"   - Logical Fails:        {fail_count} cases ({fail_percentage:.1f}%)")

    print("\n2. Top Code-Crashing Errors:")
    print("   ----------------------------------------")
    print("   The following three error types are the most frequent overall:")
    for error_name, count in top_3_errors.items():
        print(f"   - {error_name.strip()}: {count} occurrences") # Use strip() in case of leading/trailing spaces
    print(f"\n   These three errors account for {top_3_errors_percentage_of_errors:.1f}% of all code-crashing issues.")


    print("\n3. Most Problematic Questions:")
    print("   -----------------------------------------")
    print("   The following six questions generate the most problems across all users:")
    for q_name, count in top_6_questions.items():
        print(f"   - Question {q_name}: {count} problems")
    print(f"\n   These six questions collectively account for {top_6_questions_percentage_of_total:.1f}% of all logged problems.")

    if top_users_by_problems is not None:
        print("\n4. Users with the Highest Number of Problems:")
        print("   ---------------------------------------------")
        print("   The top users (anonymized IDs) with the most problem attempts are:")
        display(top_users_by_problems)

    if most_frequent_pairs is not None:
        print("\n5. Most Frequent Error Pairs Across User Profiles:")
        print("   -------------------------------------------------")
        print("   These error pairs most often appear together in the same user's error history:")
        display(most_frequent_pairs)

    if most_frequent_triplets is not None:
        print("\n6. Most Frequent Error Triplets Across User Profiles:")
        print("   ---------------------------------------------------")
        print("   These error triplets most often appear together in the same user's error history:")
        display(most_frequent_triplets)

else:
    print("--- ERROR ---")


      Comprehensive Analysis Summary

Analyzed a total of 14408 problem logs.

1. Breakdown of Problem Types:
   - Code-Crashing Errors: 5267 cases (36.6%)
   - Logical Fails:        9141 cases (63.4%)

2. Top Code-Crashing Errors:
   ----------------------------------------
   The following three error types are the most frequent overall:
   - TypeError: 2544 occurrences
   - NameError: 1247 occurrences
   - ZeroDivisionError: 607 occurrences

   These three errors account for 83.5% of all code-crashing issues.

3. Most Problematic Questions:
   -----------------------------------------
   The following six questions generate the most problems across all users:
   - Question C1-Q8: 1545 problems
   - Question C1-Q7: 1219 problems
   - Question C4-Q9: 710 problems
   - Question C1-Q1: 664 problems
   - Question C3-Q7: 664 problems
   - Question C6-Q5: 487 problems

   These six questions collectively account for 36.7% of all logged problems.

4. Users with the Highest Number of Proble

,Total_Problems,Errors,Fails
anon_id,,,
c5d3396a9af56088,118,31,87
8505438e7b8eb3e6,110,50,60
f9144dfe7f18ae59,110,107,3
337467c677292c55,105,33,72
8f2be06a802927fa,103,40,63



5. Most Frequent Error Pairs Across User Profiles:
   -------------------------------------------------
   These error pairs most often appear together in the same user's error history:


,Frequency
"(NameError, TypeError)",275
"(TypeError, ZeroDivisionError)",208
"(NameError, ZeroDivisionError)",162
"(TypeError, UnboundLocalError)",105
"(NameError, UnboundLocalError)",98



6. Most Frequent Error Triplets Across User Profiles:
   ---------------------------------------------------
   These error triplets most often appear together in the same user's error history:


,Frequency
"(NameError, TypeError, ZeroDivisionError)",153
"(NameError, TypeError, UnboundLocalError)",92
"(AttributeError, NameError, TypeError)",72
"(TypeError, UnboundLocalError, ZeroDivisionError)",66
"(NameError, UnboundLocalError, ZeroDivisionError)",59
